In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from preprocessing import merge_dfs    

In [4]:
df = merge_dfs()

# Lagre 2023 data til senere
is_2023 = "2023-01-01 00:00:00"
data_2023 = df.loc[df.index >= is_2023].drop(columns=['Trafikkmengde'])
#data_2023 = df[df['Aarstall'] == 2023].drop(columns=['Trafikkmengde'])

# Droppe kolonner der trafikkmengde er nan
data = df.loc[df.index < is_2023]
#data = df[df['Aarstall'] != 2023]
data = data.dropna(subset=['Trafikkmengde'])

# Dele i features og target, og splitte datasettet i trenings-og testdata
X = data.drop(columns=['Trafikkmengde'])
y = data['Trafikkmengde']

# Splitter dataen
X_train, X_test, y_train, y_test = train_test_split(X, y ,shuffle=False, test_size=0.3)

# Korrelasjonsmatrise
train_df = pd.concat([X_train, y_train], axis=1)

Unike verdier i "Felt": ['1' '2' 'Totalt i retning Danmarksplass' 'Totalt i retning Florida'
 'Totalt']
       Trafikkmengde
count   65250.000000
mean       50.380552
std        69.841572
min         0.000000
25%         5.000000
50%        25.000000
75%        64.000000
max       608.000000
Andel av værdata som inneholder "Relativ Luftfuktighet: 0.11075391168620219
       Globalstraling    Solskinstid  Lufttemperatur    Vindretning  \
count   707823.000000  707822.000000   707822.000000  707824.000000   
mean       157.308492      70.214651       49.650598     247.069506   
std        828.637784     827.357277      641.437364     659.156188   
min        -97.200000       0.000000      -11.400000       2.000000   
25%         -0.400000       0.000000        4.100000     146.000000   
50%          5.600000       0.000000        8.100000     164.000000   
75%        101.200000       0.000000       12.600000     298.000000   
max       9999.990000    9999.990000     9999.990000    9999.99

In [5]:
train_df

,Solskinstid,Lufttemperatur,Vindstyrke,Lufttrykk,Vindkast,Globalstraling,Vindretning,Ukedag,Maaned,Klokkeslett,Rod_dag,Trafikkmengde
Tidspunkt,,,,,,,,,,,,
2015-07-16 16:00:00,48.7,13.733333,4.333333,1014.400000,7.20,432.833333,317.500000,3,7,16,0,101.0
2015-07-16 17:00:00,60.0,13.866667,3.933333,1014.066667,6.55,378.400000,318.166667,3,7,17,0,79.0
2015-07-16 18:00:00,60.0,13.216667,4.233333,1013.966667,7.15,212.583333,319.833333,3,7,18,0,56.0
2015-07-16 19:00:00,60.0,12.683333,2.950000,1014.100000,5.45,79.750000,323.500000,3,7,19,0,45.0
2015-07-16 20:00:00,36.0,12.066667,2.483333,1014.200000,4.25,10.633333,333.500000,3,7,20,0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-07 04:00:00,0.0,11.416667,1.583333,988.866667,2.25,NaN,146.166667,2,10,4,0,1.0
2020-10-07 05:00:00,0.0,11.083333,0.900000,989.266667,1.60,NaN,132.500000,2,10,5,0,7.0
2020-10-07 06:00:00,0.0,10.833333,0.533333,989.700000,1.10,12.883333,151.166667,2,10,6,0,58.0


In [6]:
train_df.index.year

Index([2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015,
       ...
       2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020],
      dtype='int32', name='Tidspunkt', length=45669)

In [7]:
from visualizations import line_plots

In [8]:
line_plots(train_df, x="Maaned", y="Trafikkmengde", lines=train_df.index.year)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()